In [2]:
# Importações
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
import scrapy  # Scrapy importado para extração

# Inicialização do Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Função de navegação com Selenium
def navegar_para_produto(produto):
    driver.get("https://precodahora.ba.gov.br")  # URL inicial
    
    # Interações até a página de pesquisa do produto (como nos passos explicados)
    # Espera e clica nos botões necessários
    botao_tune = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "i.material-icons"))
    )
    botao_tune.click()
    time.sleep(3)
    
    botao_localizacao = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-danger.nav-location"))
    )
    botao_localizacao.click()
    time.sleep(3)
    
    botao_usar_centro = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "add-center"))
    )
    botao_usar_centro.click()
    time.sleep(3)

    campo_pesquisa = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "sbar-municipio"))
    )
    campo_pesquisa.send_keys("conqu")  # Pesquisa da cidade, por exemplo
    time.sleep(2)

    item_lista = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "set-mun"))
    )
    item_lista.click()
    time.sleep(2)
    
    botao_aplicar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "aplicar"))
    )
    botao_aplicar.click()
    time.sleep(3)

    campo_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "fake-sbar"))
    )
    campo_input.click()
    time.sleep(2)
    
    # Inserir o nome do produto e iniciar a pesquisa
    campo_input.send_keys(produto)
    time.sleep(1)
    
    botao_pesquisa = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "i.fa.fa-search"))
    )
    botao_pesquisa.click()
    time.sleep(5)  # Aguardar o carregamento dos resultados

# Função de scraping com Scrapy
class ProdutoSpider(scrapy.Spider):
    name = "produto"
    
    # URLs alvo, que podem ser preenchidas pelo Selenium
    start_urls = ["https://precodahora.ba.gov.br/produtos"]

    def parse(self, response):
        # Extração de dados específicos do produto
        for produto in response.css('div.produto'):
            yield {
                'nome': produto.css('h2.nome-produto::text').get(),
                'preco': produto.css('span.preco::text').get(),
                'detalhes': produto.css('p.detalhes::text').get(),
            }

# Loop para pesquisar e extrair dados para cada produto
produtos = ["Sicao", "Chivas", "Trident", "Ferrero"]

for produto in produtos:
    # Passo 1: Navegar até a página do produto com Selenium
    navegar_para_produto(produto)
    
    # Passo 2: Extrair dados com Scrapy
    # Inicializar e executar o spider em cada página
    from scrapy.crawler import CrawlerProcess
    
    process = CrawlerProcess()
    process.crawl(ProdutoSpider)
    process.start()


Exception ignored in: <function Service.__del__ at 0x00000235E72A98A0>
Traceback (most recent call last):
  File "c:\Users\lucaspereira\OneDrive - Cabral & Sousa Ltda\vscode\Ambiente_virtual\Projeto_scraping_preço\.venv\Lib\site-packages\selenium\webdriver\common\service.py", line 192, in __del__
    self.stop()
  File "c:\Users\lucaspereira\OneDrive - Cabral & Sousa Ltda\vscode\Ambiente_virtual\Projeto_scraping_preço\.venv\Lib\site-packages\selenium\webdriver\common\service.py", line 149, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\lucaspereira\OneDrive - Cabral & Sousa Ltda\vscode\Ambiente_virtual\Projeto_scraping_preço\.venv\Lib\site-packages\selenium\webdriver\common\service.py", line 129, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\lucaspereira\AppData\Local\anaconda3\Lib\urllib\request.py", line 215, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 